<a href="https://colab.research.google.com/github/Toroto006/iml2021/blob/main/task1B/Task1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Trying to solve Task 1B now!

In [178]:
import pandas as pd
import numpy as np

trainUrl = 'https://raw.githubusercontent.com/Toroto006/iml2021/main/task1B/train.csv?token=AH44KNVSZQ2TFA45ZGBDBMDAMV2JK'

#pd.set_option("precision", 20)
trainPD = pd.read_csv(trainUrl, float_precision=None)
#trainPD

Some trying on the precision of pandas.
Looks like the float_precision="high" should be enough for this case.

In [179]:
#print(f"precision 20: %10.8e" % (0.43000000000000010436 - 0.43000000000000005))
#print(f"precision 22: %10.8e" % (0.4300000000000001043610 - 0.43000000000000005))
#print(f"high precision: %10.8e and %10.8e" % (0.42999999999999999334 - 0.43000000000000005, 0.0800000000000000016653 - 0.08000000000000007))

Create cross validation folds:

In [180]:
from sklearn.model_selection import KFold

splits = 7

X = trainPD.loc[:,'x1':'x5']
Y = trainPD['y']
test = {}
train = {}
i = 0
# maybe use seed and random shuffle
kf = KFold(n_splits=splits, random_state=None, shuffle=False)
for train_index, test_index in kf.split(trainPD):
  train[i] = (X.iloc[train_index].to_numpy(), Y.iloc[train_index].to_numpy())
  test[i] = (X.iloc[test_index].to_numpy(), Y.iloc[test_index].to_numpy())
  i += 1
#test(X_test, y_test)

Creating feature set according to our exercise.

In [181]:
feature_list = [lambda x: x,            # Linear
                lambda x: np.square(x), # Quadratic 
                lambda x: np.exp(x),    # Exponential
                lambda x: np.cos(x)]    # Cosine
                #lambda x: 1]       # Constant
               
def build_features(x_train):
    d3_phi = np.stack(tuple(feature(x_train) for feature in feature_list)) # create the feature stack
    first4 = d3_phi.transpose(1,0,2).reshape(d3_phi.shape[1],-1) # reshape into 600, 20
    final = np.ones((first4.shape[0],first4.shape[1]+1))  
    final[:,:-1] = first4 # add missing constant feature by replacing all other ones with features
    return final

In [182]:
# Ridge regression with small lambda still linear regression?
def get_coefficients(x_train, y_train, lamb=1e-16):
    phi_train = build_features(x_train)
    dim = phi_train.shape[-1]
    w_hat = np.dot(np.linalg.pinv(np.dot(phi_train.T, phi_train) + lamb * np.eye(dim)), 
                   np.dot(phi_train.T, y_train))
    return w_hat

In [183]:
def evaluate_features(x_test, y_test, w_hat):
    phi_test = build_features(x_test)
    error = np.sqrt(np.power(y_test - phi_test @ w_hat, 2).mean())
    return error 

Try to do the evaluation

In [184]:
w_hat = get_coefficients(train[0][0], train[0][1])
#print(w_hat)
evalFeatures = evaluate_features(test[0][0], test[0][1], w_hat)
# evalFeatures

Doing it by hand for one seemed like a good prediction, lets do it for all folds and see the avg error.

In [185]:
errors = []
for i in range(0, splits):
  w_hat = get_coefficients(train[i][0], train[i][1])
  fold_step_err = evaluate_features(test[i][0], test[i][1], w_hat)
  errors.append(fold_step_err)
print(f"Errors over folds: %s" % errors)
print(f"Average error over all folds: %10.8e" % (np.average(errors)))

Errors over folds: [1.8471530334843937, 2.0300637018802674, 1.9766852150545569, 1.8698227492589279, 2.131994063671641, 2.8415863849203107, 2.1121027335860183]
Average error over all folds: 2.11562970e+00


Over the folds does not look so bad, so let's do one final train over the whole training set:

In [186]:
w_hat_final = get_coefficients(X.to_numpy(), Y.to_numpy())
#w_hat_final

Output the data for submission.

In [187]:
res = pd.DataFrame(data=w_hat_final)
res.to_csv("out.csv", index=False, header=False, float_format='%.22f')
#res